In [1]:
import radar
%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np
import importlib
import cfar
import tracking

In [2]:
importlib.reload(radar)
importlib.reload(cfar)

<module 'cfar' from 'c:\\Users\\Isabel\\Documents\\simulador_OTH\\cfar.py'>

In [3]:
#cod1=radar.Codigo_pseudoaleatorio(13,30,1)
cod1=radar.Codigo_pseudoaleatorio(semilla = 202305041529+4,numero_bits=22,numero_muestras_bit=1)

In [44]:
def cod_y_correlacion():
    plt.figure(1,figsize=(10,6),dpi=100,layout='tight')
    plt.clf()
    plt.subplot(2,1,1)
    plt.title("Pulso codificado (secuencia pseudoaleatoria)")
    ycod=cod1.muestras_codigo
    xs=np.arange(-len(ycod)+1,len(ycod))
    ycorr=np.correlate(ycod,ycod,"full")
    plt.stem(xs,np.concatenate((np.zeros_like(ycod[:-1]),ycod)))
    plt.subplot(2,1,2)
    plt.title("Autocorrelación de Pulso codificado")
    plt.stem(xs,ycorr)
    plt.plot(xs,ycorr,'b--')
    plt.xlabel("lag")
    plt.show()
cod_y_correlacion()

In [4]:
rad1 = radar.Canal_radar(frec_muestreo=200000,frec_portadora=7e6,pulso=cod1.muestras_codigo,guarda=15000,rango_maximo=400e3, semilla_rng=202305041529+4)
comp1 = radar.Compresor_pulso(rad1)
simulador = radar.Simulador_escenario(rad1)
escenario = [radar.mru(140000,-4), radar.mru(75000,12), radar.mru(100000,-6), radar.mru(55000,0)]
exploraciones = simulador.exploracion(escenario,0,60,120, snr_rango_max=1/3)

In [19]:
#plot de amplitud cuadrada de la correlación para algunos tiempos en función del rango
def fig14():
  plt.figure(figsize=(12,20),dpi=100)
  plt.title("Amplitud cuadrada de la correlación a diferentes tiempos")
  #plt.xlabel("Rango (en km)")
  #plt.ylabel("Amplitud")
  for i in range(10):
    plt.subplot(10,1,i+1)
    plt.xlabel("Rango (en km)")
    plt.ylabel("Amplitud")
    plt.plot(comp1.amplitud_cuadrada_correlacion(exploraciones[i*10]))
  plt.show()
fig14()

In [18]:
def fig15(compresor_prueba,exploraciones):
  correlacion = compresor_prueba.amplitud_cuadrada_correlacion(exploraciones[30])
  radar ={"frec_muestreo":compresor_prueba.frec_muestreo, # en 1/s
          "ancho_pulso":compresor_prueba.pulso.size, # adimensional
          "distancia_ciega": compresor_prueba.rango_minimo_km, # km
          "rango_por_unidad_de_retardo":1.5e5} # km/s
  detector={"ancho_ventana":30,
            "k":(30*6)//7}
  detector["T"]=cfar.calcula_t_para_pfa(detector["ancho_ventana"],detector["k"])
  rangos,detalle= cfar.detectarOsCfar(radar,correlacion,detector)
  plt.figure(figsize=(16,6),dpi=150)
  plt.subplot(2,1,1)
  plt.plot(correlacion,'.--')
  detecciones = detalle["posiciones"]
  plt.plot(detecciones,correlacion[detecciones],'r+')
  plt.plot(detalle["centros"],detalle["umbrales"],'.--')
  plt.subplot(2,1,2)
  plt.plot(rangos,np.ones_like(rangos),'ro')
  #plt.xlim(0,100)
  plt.show()
fig15(comp1,exploraciones)

In [7]:
plt.subplot(2,1,1)
plt.plot(np.real(exploraciones[100]))
plt.subplot(2,1,2)
plt.plot(np.imag(exploraciones[100]))

In [8]:
sim = simulador.simula(escenario,0,.5,120,120)

In [16]:
def fig16(exploracion,comp_pulso):
    plt.figure(16,figsize=(10,10))
    corr =  np.array([comp_pulso.amplitud_cuadrada_correlacion(exploracion[i,:]) for i in range(exploracion.shape[0])])
    corr = np.fft.fftshift(corr,0)
    R,F = np.meshgrid(np.arange(corr.shape[1]),np.fft.fftshift(np.fft.fftfreq(corr.shape[0],0.5)))
    ax=plt.axes(projection='3d')
    ax.plot_surface(R,F,corr)
    plt.show()
fig16(sim[100],comp1)

In [ ]:
535-514

In [10]:
exploraciones.shape

(120, 514)

In [11]:
# kf = tracking.KF(initial_x=0.0, initial_v=1.0, accel_variance=0.1)
# kf = KF(initial_x=0.0, initial_v=1.0, accel_variance=0.1)

def fn1(compresor,exploraciones,dt=0.1,varDet=.1,varAcel=.1,tol=1,tolPred=10):
    radar ={"frec_muestreo":compresor.frec_muestreo, # en 1/s
        "ancho_pulso":compresor.pulso.size, # adimensional
        "distancia_ciega": compresor.rango_minimo_km, # km
        "rango_por_unidad_de_retardo":1.5e5} # km/s
    detector={"ancho_ventana":30,
                "k":(30*6)//7}
    detector["T"]=cfar.calcula_t_para_pfa(detector["ancho_ventana"],detector["k"])
    seguimiento = tracking.GestorDeTrayectos()
    for i in range(exploraciones.shape[0]):
        a2 = compresor.amplitud_cuadrada_correlacion(exploraciones[i,:])
        rangos,_ = cfar.detectarOsCfar(radar,a2,detector)
        seguimiento.actualizaTrayectos(rangos,dt,varDet,varAcel,tol,tolPred)
    return seguimiento.copiaTrayectos()


trayectos=fn1(comp1,exploraciones,varDet=1,varAcel=1,tol=3)


In [ ]:
plt.figure(1)
for trayecto in trayectos:
    datos_trayecto = np.asanyarray(trayecto)
    t = datos_trayecto[:,0]
    x = datos_trayecto[:,1]
    var_x= datos_trayecto[:,2]
    v= datos_trayecto[:,3]
    var_v= datos_trayecto[:,4]
    plt.plot(t,x)
plt.show()


In [13]:
np.asanyarray(trayectos[3])

array([[ 1.00000000e-01,  9.97500000e+01,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00],
       [ 2.00000000e-01,  9.97500000e+01,  7.08867936e-01,
         0.00000000e+00,  1.00248445e+00],
       [ 3.00000000e-01,  9.94925887e+01,  5.85845583e-01,
        -7.43793612e-02,  1.00000016e+00],
       [ 4.00000000e-01,  9.93533317e+01,  5.21255621e-01,
        -1.44929992e-01,  9.90436241e-01],
       [ 5.00000000e-01,  9.92585319e+01,  4.86832482e-01,
        -2.08016029e-01,  9.72384811e-01],
       [ 6.00000000e-01,  9.90194265e+01,  4.70122995e-01,
        -4.24406698e-01,  9.45473077e-01],
       [ 7.00000000e-01,  9.86595156e+01,  4.63621252e-01,
        -7.82662956e-01,  9.10481040e-01],
       [ 8.00000000e-01,  9.83501917e+01,  4.62271631e-01,
        -1.05978855e+00,  8.69156272e-01],
       [ 1.00000000e+00,  9.77808544e+01,  5.07380216e-01,
        -1.48180778e+00,  8.06833240e-01],
       [ 1.10000000e+00,  9.74162850e+01,  4.95127634e-01,
        -1.72806349e+00